# Import Required Libraries

In [ ]:
import sys

# Data Manipulation libraries:
import numpy as np
import pandas as pd
from copy import deepcopy

# Plotting libraries
import mplsoccer
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

[mplsoccer docs](https://mplsoccer.readthedocs.io/en/latest/)

# Load the Data

**Data source** --> Statsbomb \
**Competition** --> La Liga \
**Season** --> 2019-20

Folder Structure

- FA Masterclass
    - notebooks
    - data
    - reports
    - docs

In [ ]:
eventsDataLaLiga2019 = pd.read_csv("../data/matchwise_events_data.csv", low_memory=False)

In [ ]:
seasonMetaDataLaLiga2019 = pd.read_csv("../data/season_matchwise_data.csv", low_memory=False)

In [ ]:
pd.set_option("display.max_rows", 150)
pd.set_option("display.max_columns", 50)

In [ ]:
eventsDataLaLiga2019.columns

# Know Your Data (KYD)

[StatsBomb Data Docs](https://github.com/statsbomb/open-data)

# Data Cleaning and Exploratory Data Analysis (EDA)

EDA, mainly, does three things:

1. It helps clean up a dataset.
1. It helps identify metrics that can be added by calculations.
2. It gives you a better understanding of the variables and the relationships between them.

## Get Meta Info (Event Data)

In [ ]:
eventsDataLaLiga2019.info()

In [ ]:
eventsDataLaLiga2019.columns

In [ ]:
eventsDataLaLiga2019.columns.values

In [ ]:
eventsDataLaLiga2019.dtypes

In [ ]:
eventsDataLaLiga2019.head(10)

In [ ]:
eventsDataLaLiga2019[["type.id", "type.name"]].drop_duplicates()

In [ ]:
eventPassdf = deepcopy(eventsDataLaLiga2019[eventsDataLaLiga2019["type.id"] == 30])

In [ ]:
eventPassdf.shape

## Get Meta Info (Season Data)

In [ ]:
seasonMetaDataLaLiga2019.info()

## Join Season Data to Event Data

In [ ]:
seasonMetaDataLaLiga2019.columns

In [ ]:
seasonCols = ['match_id', 'match_date', 'kick_off', 'home_score', 'away_score',
              'home_team.home_team_id', 'home_team.home_team_name',
              'away_team.away_team_id', 'away_team.away_team_name']

In [ ]:
seasonCols.__len__()

In [ ]:
seasonMetaDataLaLiga2019[seasonMetaDataLaLiga2019["match_id"] == 303731]

In [ ]:
pd.merge(eventsDataLaLiga2019,
                       seasonMetaDataLaLiga2019[seasonCols],
                       how="left", on=["match_id"])

In [ ]:
eventsDataLaLiga2019 = pd.merge(eventsDataLaLiga2019,
                                seasonMetaDataLaLiga2019[seasonCols],
                                how="left", on=["match_id"])

## Feature Engineering

### `started` Column

In [ ]:
eventsDataLaLiga2019.head()

In [ ]:
eventsDataLaLiga2019[(eventsDataLaLiga2019["match_id"] == 303707)].head()

In [ ]:
eventsDataLaLiga2019[(eventsDataLaLiga2019["match_id"] == 303707)
                    & (eventsDataLaLiga2019["type.id"] == 35)]["tactics.lineup"]

In [ ]:
eventsDataLaLiga2019[(eventsDataLaLiga2019["match_id"] == 303707)
                    & (eventsDataLaLiga2019["type.id"] == 35)]["tactics.lineup"].iloc[0]

In [ ]:
pd.json_normalize(eventsDataLaLiga2019[(eventsDataLaLiga2019["match_id"] == 303707)
                    & (eventsDataLaLiga2019["type.id"] == 35)]["tactics.lineup"].iloc[0])

In [ ]:
from ast import literal_eval

In [ ]:
eventsDataLaLiga2019[(eventsDataLaLiga2019["match_id"] == 303707)
                    & (eventsDataLaLiga2019["type.id"] == 35)]["tactics.lineup"].apply(literal_eval).iloc[0]

In [ ]:
pd.json_normalize(eventsDataLaLiga2019[(eventsDataLaLiga2019["match_id"] == 303707)
                    & (eventsDataLaLiga2019["type.id"] == 35)]["tactics.lineup"].apply(literal_eval).iloc[1])

In [ ]:
df = pd.json_normalize(eventsDataLaLiga2019[(eventsDataLaLiga2019["match_id"] == 303707)
                    & (eventsDataLaLiga2019["type.id"] == 35)]["tactics.lineup"].apply(literal_eval).iloc[0])

In [ ]:
df["started"] = "Yes"
df

In [ ]:
df.insert(0, "match_id", 303707)

In [ ]:
df

In [ ]:
finalLineUpdf = pd.DataFrame()
for m_id in eventsDataLaLiga2019["match_id"].unique():
    print(m_id, "\n")
    match_df = eventsDataLaLiga2019[(eventsDataLaLiga2019["match_id"] == m_id)
                                    & (eventsDataLaLiga2019["type.id"] == 35)]["tactics.lineup"].apply(literal_eval)
    
    df1 = pd.json_normalize(match_df.iloc[0])
    df2 = pd.json_normalize(match_df.iloc[1])

    df = df1.append(df2)
    df.insert(0, "match_id", m_id)
    
    df["started"] = "Yes"
    
    finalLineUpdf = finalLineUpdf.append(df)

print(finalLineUpdf.head(20))

In [ ]:
finalLineUpdf

In [ ]:
finalLineUpdf.columns

In [ ]:
eventsDataLaLiga2019 = pd.merge(eventsDataLaLiga2019,
                                finalLineUpdf[['match_id', 'player.id', 'jersey_number', 'started']],
                                how="left",
                                on=['match_id', 'player.id'])

In [ ]:
eventsDataLaLiga2019.shape

In [ ]:
eventsDataLaLiga2019[(eventsDataLaLiga2019["match_id"] == 303421)
                    & (eventsDataLaLiga2019["player.id"] == 6571)]

### `minsPlayed` Column

In [ ]:
minsPlayeddf = eventsDataLaLiga2019.groupby(["match_id", "player.id"]).agg(
    {"player.name": "first", "team.name": "first", "minute": ["min", "max"]}).reset_index()
minsPlayeddf.head()

In [ ]:
eventsDataLaLiga2019[eventsDataLaLiga2019['substitution.outcome.id'].notnull()]\
[["match_id", "player.name", 'substitution.replacement.id', 'substitution.replacement.name', "minute"]]

In [ ]:
subData = eventsDataLaLiga2019[["match_id", "substitution.replacement.id", "minute"]]\
.dropna(subset=["substitution.replacement.id"])

In [ ]:
subData.rename(columns={"minute": "minMinute",
                        "substitution.replacement.id": "subID"}, inplace=True)

In [ ]:
subData.head(10)

In [ ]:
pd.merge(eventsDataLaLiga2019, subData, how="left",
        left_on=["match_id", "player.id"], right_on=["match_id", "subID"])

In [ ]:
eventsDataLaLiga2019 = pd.merge(eventsDataLaLiga2019, subData, how="left",
                       left_on=["match_id", "player.id"],
                       right_on=["match_id", "subID"])

In [ ]:
eventsDataLaLiga2019[eventsDataLaLiga2019["match_id"] == 303421]

In [ ]:
eventsDataLaLiga2019.dropna(subset=["started"])["minMinute"].unique()

In [ ]:
eventsDataLaLiga2019[eventsDataLaLiga2019['substitution.outcome.id'].notnull()]\
[["match_id", "player.id", "player.name", "minute"]]

In [ ]:
subOutData = eventsDataLaLiga2019[eventsDataLaLiga2019['substitution.outcome.id'].notnull()]\
            [["match_id", "player.id", "minute"]]

In [ ]:
subOutData.rename(columns={"minute": "maxMinute"}, inplace=True)

In [ ]:
pd.merge(eventsDataLaLiga2019, subOutData,
         how="left",
         on=["match_id", "player.id"])

In [ ]:
eventsDataLaLiga2019 = pd.merge(eventsDataLaLiga2019, subOutData,
         how="left",
         on=["match_id", "player.id"])

In [ ]:
eventsDataLaLiga2019.dropna(subset=["maxMinute"])

In [ ]:
eventsDataLaLiga2019.groupby(["match_id"])["minute"].transform(lambda x: x.max())

In [ ]:
eventsDataLaLiga2019["maxMinsMatch"] =\
    eventsDataLaLiga2019.groupby(["match_id"])["minute"].transform(lambda x: x.max())

In [ ]:
eventsDataLaLiga2019["minMinute"] = np.where(eventsDataLaLiga2019["started"] == "Yes",
                                             0, eventsDataLaLiga2019["minMinute"])

In [ ]:
eventsDataLaLiga2019["maxMinute"] = np.where(eventsDataLaLiga2019["maxMinute"].isnull(),
                                             eventsDataLaLiga2019["maxMinsMatch"],
                                             eventsDataLaLiga2019["maxMinute"])

In [ ]:
eventsDataLaLiga2019.groupby(["match_id", "player.id"]).agg(
    {"player.name": "first", "team.name": "first",
     "minMinute": "first", "maxMinute": "max"}).reset_index()

In [ ]:
eventsDataLaLiga2019.groupby(["match_id", "player.id"]).agg(
    {"player.name": "first", "team.name": "first",
     "minute": ["min", "max"]}).reset_index()

In [ ]:
minsPlayeddf = eventsDataLaLiga2019.groupby(["match_id", "player.id"]).agg(
    {"player.name": "first", "team.name": "first",
     "minMinute": "first", "maxMinute": "max"}).reset_index()
minsPlayeddf.head()

In [ ]:
minsPlayeddf["minsPlayed"] = minsPlayeddf["maxMinute"].subtract(minsPlayeddf["minMinute"])

In [ ]:
eventsDataLaLiga2019 = pd.merge(eventsDataLaLiga2019,
                       minsPlayeddf[["match_id", "player.id", "minsPlayed"]],
                       how="left",
                       on=["match_id", "player.id"])
eventsDataLaLiga2019.head()

In [ ]:
eventsDataLaLiga2019.head(10)

In [ ]:
eventsDataLaLiga2019.to_csv("../data/matchwise_events_data_updated.csv", index=False)